In [1]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from statistics import mean
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
from rlhfutils.dproc_utils import process_df_ultra, procmean, create_pairwise_dataframe
from datasets import Dataset
from IPython.core.display import display, HTML
from rlhfutils.data import load_ultra, tokenize_dset
import pickle

/u/prasanns/miniconda3/envs/rlhfenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2400406/3997923612.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
_, evd = load_ultra("../../data/synthnoun90/")

Loading cached shuffled indices for dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-8bed4f2ea98bc676.arrow


initial size  20000
18000
eval len
2000


In [12]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

In [13]:
def add_row_index(example, idx):
    example['row_index'] = idx
    return example
    
evals = evd.map(add_row_index, with_indices=True)

_, evals = tokenize_dset(evals, evals, None, tokenizer)

Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-fa8394ab62801281.arrow
Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-a40925c7574ea649_*_of_00010.arrow
Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-a9ad5acbceed9faa.arrow
Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-eaea545c2c4250ef_*_of_00010.arrow
Loading cached processed dataset at /scratch/cluster/prasanns/research/rlhf-length-biases/data/synthnoun90/cache-06110f01fde954d3.arrow


In [14]:
andf = pd.DataFrame(evd[evals['ids']])

In [16]:
with open("../../synthnoun90nomag.pickle", "rb") as f:
    mpreds = pickle.load(f)[0]
    andf['rewj'] = mpreds[0]
    andf['rewk'] = mpreds[1]

In [17]:
len(andf[andf.rewj>andf.rewk])/len(andf)

0.7031484257871065

In [18]:
#ms = ["mn", "ifg", "hf", "tn", "hn",]
ms = ["nouns", "tokens"]
for metr in ms: 
    print(metr)
    print(len(andf[(andf.rewj>andf.rewk)==(andf[metr+"_j"]>andf[metr+"_k"])])/len(andf))

nouns
0.6829085457271364
tokens
0.29910044977511246


In [52]:
andf['tok_j'] = andf['tokj']
andf['tok_k'] = andf['tok']

In [19]:
ms = ["mn", "ifg", "hf", "tn", "hn","tok"]
ms = ["nouns", "tokens", "score"]

for metr in ms: 
    print(metr)
    rlist = list(andf.rewj)+list(andf.rewk)
    glist = list(andf[metr+"_j"])+list(andf[metr+"_k"])
    #glist = list(andf["tokj"])+list(andf["tok"])
    
    nrl = []
    nrg = []
    for i in range(len(glist)):
        try: 
            nrg.append(float(glist[i]))
            nrl.append(float(rlist[i]))
        except: 
            ""
    
    print(pearsonr(nrl, nrg))
    print(spearmanr(list(andf.rewj)+list(andf.rewk), list(andf[metr+"_j"])+list(andf[metr+"_k"])))

nouns
PearsonRResult(statistic=-0.11595972752422351, pvalue=1.888723796002985e-09)
SignificanceResult(statistic=-0.16433595809661336, pvalue=1.3128685409249314e-17)
tokens
PearsonRResult(statistic=-0.26874470601591643, pvalue=2.2878992517635228e-45)
SignificanceResult(statistic=-0.3168752746406668, pvalue=2.6800978739145064e-63)
score
PearsonRResult(statistic=0.3407801394095387, pvalue=1.5497819076137554e-73)
SignificanceResult(statistic=0.3429394373738802, pvalue=1.6586583704788754e-74)


In [59]:
a[1].shape

(1938, 1)

In [34]:
evd

,question,source,modj,modk,tokj,tok,response_j,response_k,magnitude,mn_j,mn_k,hf_j,hf_k,hn_j,hn_k,tn_j,tn_k,ifg_j,ifg_k
0,"In this task, you are given two sets, and a qu...",flan_v2_niv2,ultralm-65b,llama-2-70b-chat,3,83,No.,"No, the element '9' is not in the intersection...",2.50,4.00,1.50,3,1,5,1,5,2,3,2
1,"As a researcher, you are tasked with analyzing...",evol_instruct,wizardlm-70b,llama-2-13b-chat,554,684,"Certainly, the civilization you're referring t...",Thank you for your comprehensive question! I'l...,0.25,4.75,4.50,4,4,5,5,5,5,5,4
2,"Given the task definition, example input & out...",flan_v2_niv2,ultralm-65b,llama-2-7b-chat,219,154,"The short answer to the question ""1 acre can b...",Thank you for the task! I'm happy to help you ...,2.50,4.25,1.75,4,1,5,2,5,3,3,1
3,"A, B, and 14 other people participate in a tab...",sharegpt,ultralm-13b,wizardlm-13b,402,525,"First, we need to determine the probability of...",Given that A has a 0% winning rate against B a...,0.00,1.75,1.75,1,1,1,2,3,2,2,2
4,"Teacher: In this task, you are given a sentenc...",flan_v2_niv2,wizardlm-13b,wizardlm-70b,143,300,"9. ""Toutes les plantes respirent par les stigm...","I apologize, but the problem statement you pro...",0.75,3.75,3.00,2,3,5,4,3,2,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3833,Please answer the following question: Generate...,flan_v2_p3,alpaca-7b,ultralm-65b,65,162,I got this app to get my daily updates about t...,"As an AI assistant, I do not have personal opi...",0.00,4.50,4.50,3,3,5,5,5,5,5,5
3834,Write a steamy romance novel about two adventu...,ultrachat,vicuna-33b,llama-2-7b-chat,927,262,Title: Whirlwind in the Wanderlust\n\nChapter ...,I'm glad you're interested in writing a romanc...,2.25,5.00,2.75,5,1,N/A,5,5,3,5,2
3835,How can I create a Scala function that convert...,evol_instruct,wizardlm-7b,alpaca-7b,279,47,That's correct! The formula to convert Celsius...,def celsiusToFahrenheit(celsius: Double): Doub...,2.00,4.50,2.50,4,1,5,1,5,3,4,5
3836,You will be given a definition of a task first...,flan_v2_niv2,llama-2-70b-chat,gpt-3.5-turbo,112,2,"Yes.\n\nExplanation:\n\nThe Head event ""Person...",No,2.50,4.50,2.00,4,3,5,1,5,1,4,3
